# Problema 1: Control Planta hidráulica

### Control manual:

Como se explicó en la sección anterior, las ecuaciones que modelan la central hidraúlica son las siguientes:

$$L=\frac{h}{\tan(\theta)}$$


$$V_{h}(t)=\frac{bh^{2}(t)}{2}+\frac{h^{3}(t)}{3}$$


$$\frac{dV_{h}}{dt}=bh(t)\frac{dh}{dt}+h^{2}(t)\frac{dh}{dt}=q_{in}-Cab_{1}\sqrt{gh(t)}$$


$$ P=\eta \gamma Cab_{1}\sqrt{h(t)}\left(h(t) + \frac{v_2^2}{2g}-64\cdot\frac{\mu L_{t} v_2}{\rho D^2 2g}\right) $$


En este primer ejercicio, se busca que mediante prueba y error, se cambie manualmente el parámetro $b_{1}$ que es la apertura de la compuerta tipo Stoney de manera que a los 6 segundos la potencia generada sea 50 MW, considerando una eficiencia de la turbina del 90%. El valor de los demás parámetros se encuentra en el código. 

In [7]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox
import numpy as np
import numpy
import matplotlib.pyplot as plt
from IPython.display import display
%matplotlib inline
import math
from scipy.integrate import odeint




# function that returns dy/dt
def model(z,t,qin,C,a,b1,g,Lt,b,mu,rho,D,hi):
    h=z[0]                                                #Funcion EDO
    Q = C*a*b1*math.sqrt(g*(hi+h))                        #Caudal de salida
    v_2 = 4*Q/(math.pi*D**2)                              #Velocidad de salida
    dhdt = (qin-Q)/(b*h+h**2) #EDO para altura
    return [dhdt]

def verificar(bb):
    b1=float(text1.value)
    vector_potencias,vector_b1,vector_tiempo=potencia(qin,C,a,b1,b1max,b,efic,g,Lt,Le,mu,rho,D,hi)
    line1.set_ydata(np.dot(vector_potencias,1/1000000))
    line2.set_ydata(vector_b1)
    pfinal=vector_potencias[-1]/1000000
    pfinal=int(pfinal*100)/100.0
    lbl4.value='Potencia= '+str(pfinal)+' MW'
    
@widgets.interact_manual(
    color=['blue', 'red', 'green'], b1=(0., 10.))
def plot(b1=1., color='blue', grid=True):
    t = np.linspace(0., +6., 1000)
    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    
        #Parametros de tiempo:
    h0 = 155    #initial condition
    Time = 6    #Tiempo de simulacion
    dt = 0.01   #Periodo de muestreo
    t = np.arange(Time+1)

    #Parametros del embalse:
    qin = 100+ 30*np.cos(2*math.pi/Time*dt*10)   #Caudal de entrada
    C = 0.64    #Parametro 
    a = 1       #Ancho de la compuerta
    b1max = 10  #Apertura maxima vertical de la compuerta
    b = 100     #Base de la seccion transversal del estanque
    efic  = 0.9 #Eficiencia de la turbina
    g = 9.81    #Aceleracion de gravedad
    Lt = 200    #distancia a la que está la turbina por la tubería
    Le = 150    #distancia de   
    mu = 0.001  #mu del agua 
    rho = 1000  #densidad del agua
    D = 5       #diametro de la tuberia
    hi = 20     #altura desde la base del embalse

    #Vectores de informacion:
    vector_potencias = [] 
    vector_b1 = []
    vector_h = []
    vector_qin = []
    vector_tiempo = []

    #Inician variables iniciales:
    Pinicial = 0
    hinicial = h0
    vector_potencias.append(Pinicial)
    vector_h.append(hinicial)
    vector_b1.append(b1)
    vector_qin.append(qin)
    vector_tiempo.append(0)
    k=0

    for i in np.arange(0,Time,dt):

        #Se define el caudal de entrada del embalse y el error:
        qin = 100 + 30*np.cos((2*math.pi/Time)*i*10) 

        #Se calcula la EDO y guardan outputs de tiempo y altura:
        out = odeint(model,hinicial,[i,i+dt],args=(qin,C,a,b1,g,Lt,b,mu,rho,D,hi))
        vector_tiempo.append(i+dt)
        hinicial = out[:,0][-1]

        #Calculamos caudal, velocidad de salida y perdidas:
        Q = C*a*b1*math.sqrt(g*hi)
        v_2 = 4*Q/(math.pi*D**2)
        hf = 64*(mu/rho)*(Lt/D**2)*(v_2/(2*g))

        #Se calcula la potencia de la iteración:
        P = rho*g*Q*(hi+hinicial-(v_2**2/(2*g))-hf)
        P = efic*P
        Pinicial = P

        #Se guardan los vectores:
        vector_potencias.append(Pinicial) 
        vector_h.append(hinicial)
        vector_qin.append(qin)
        vector_b1.append(b1)
        k=k+1
    
    ax.plot(vector_tiempo,np.dot(vector_potencias,1/1000000),'b-',linewidth=2,color=color)
    ax.grid(grid)
    plt.title('Potencia [MW]')
    pfinal=vector_potencias[-1]/1000000
    pfinal=int(pfinal*100)/100.0
    print('Potencia: '+str(pfinal)+' MW')
    
    

interactive(children=(FloatSlider(value=1.0, description='b1', max=10.0), Dropdown(description='color', option…

### Control automático:

En la realidad podría resulta difícil encontrar la apertura adecuada, sobre todo si se desean realizar cambios de potencia a lo largo del tiempo y estos tienen que ser exactos. Para facilitar el control de la central, se utiliza un controlador PID (Proporcional Integral y Derivativo) sobre la hidroeléctrica. Un controlador tiene la finalidad de llevar el sistema a un punto de operación deseado, usualmente a este punto se llama referencia y en este caso la variable que se desea manejar como referencia es la potencia de la turbina. El controlador PID realiza los cálculos necesarios para llevar el sistema hacia la referencia manejando su variable manipulada, en este caso la variable manipulada es la apertura de la compuerta. Es decir, este programa manipula la apertura de la compuerta para lograr la potencia de referencia definida.


En el siguiente código se implementa el controlador que nos permite fijar una referencia en la potencia (potencia que queremos alcanzar) y el programa controlara la apertura de la compuerta (variable manipulada) con la intención de alcanzar dicha referencia de potencia. 


Este controlador tiene un rango de operación de $0 [MW]$ a $80 [MW]$, mientras que la apertura de la compuerta puede ir de 0 metros a 10 metros. Para poder poner a prueba el controlador se establecen 3 referencias distintas las cuales se pueden variar de manera arbitraria (se pueden modificar) para mostrar la apertura de compuerta necesaria para alcanzar cierta potencia. La referencia 1 $Pref_1$ estará activa en el intervalo de tiempo $[0s-2s]$, la referencia 2 $Pref_2$ estará activa en el intervalo de tiempo $[2s-4s]$ mientras que la referencia 3 $Pref_3$ estará activa en el intervalo de tiempo $[4s-6s]$. En el grafico final veremos cómo cambia la dinámica de la potencia y la apertura de la compuerta para lograr las distintas potencias establecidas. Además se consideró $\theta=\frac{\pi}{4}$.

Las referencias se pueden variar son sliders que van de 0 a 80 MW.

In [8]:
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import numpy as np
import numpy
import math

# function that returns dy/dt
def model(z,t,qin,C,a,b1,g,Lt,b,mu,rho,D,hi):
    h=z[0]                                                #Funcion EDO
    Q = C*a*b1*math.sqrt(g*(hi+h))                        #Caudal de salida
    v_2 = 4*Q/(math.pi*D**2)                              #Velocidad de salida
    dhdt = (qin-Q)/(b*h+h**2) #EDO para altura
    return [dhdt]

vector_potencias = [] 
vector_b1 = []
vector_h = []
vector_qin = []
vector_tiempo = []
vector_error = []


@widgets.interact_manual(
    color=['blue', 'red', 'green'], Pref1=(0., 80.), Pref2=(0., 80.), Pref3=(0., 80.))
def plot(Pref1=10,Pref2=10,Pref3=10, color='blue'):

    Pref1 = 1000000*float(Pref1)
    Pref2 = 1000000*float(Pref2)
    Pref3 = 1000000*float(Pref3)
    
    t = np.linspace(0., +6., 1000)
    #Parametros de tiempo:
    h0 = 155    #initial condition
    Time = 6    #Tiempo de simulacion
    dt = 0.01   #Periodo de muestreo
    t = np.arange(Time+1)

    #Parametros del embalse:
    qin = 100+ 30*numpy.cos(2*math.pi/Time*dt*10)   #Caudal de entrada
    C = 0.64    #Parametro 
    a = 1       #Ancho de la compuerta
    b1 = 0      #Apertura vertical de la compuerta
    b1max = 10  #Apertura maxima vertical de la compuerta
    b = 100     #Base de la seccion transversal del estanque
    efic  = 0.9 #Eficiencia de la turbina
    g = 9.81    #Aceleracion de gravedad
    Lt = 200    #distancia a la que está la turbina por la tubería
    Le = 150    #distancia de   
    mu = 0.001  #mu del agua 
    rho = 1000  #densidad del agua
    D = 5       #diametro de la tuberia
    hi = 20     #altura desde la base del embalse

    #Parametros del controlador:
    Kd = 0              #Ganancias del PID
    Ki = 0.00000007  
    Kp = 0.000000001 
    Pref = Pref1        #Potencia de referencia del controlador [0MW-80MW]


    #Vectores de informacion:
    vector_potencias = [] 
    vector_b1 = []
    vector_h = []
    vector_qin = []
    vector_tiempo = []
    vector_error = []

    #Inician variables iniciales:
    Pinicial = 0
    hinicial = h0
    bl = 0
    error = Pref - Pinicial 
    vector_potencias.append(Pinicial)
    vector_h.append(hinicial)
    vector_b1.append(b1)
    vector_qin.append(qin)
    vector_tiempo.append(0)
    vector_error.append(error)
    I = 0
    k = 0

    for i in numpy.arange(0,Time,dt):
        if i>2:
            if i>4:
                Pref = Pref3
            else:
                Pref = Pref2
        else:
            Pref = Pref1

        #Se define el caudal de entrada del embalse y el error:
        qin = 100 + 30*numpy.cos((2*math.pi/Time)*i*10) 
        error_old = error
        error = Pref - Pinicial

        #Se aplica el PID:
        P = Kp*error
        I = I+Ki*error
        D1 = Kd*(error-error_old)/dt
        b1 = P+I+D1

        #Se aplican limites fisicos de la compuerta:
        if b1>b1max:
            b1 = b1max

        elif b1<0:
            b1 = 0

        else:
            b1 = b1

        #Se calcula la EDO y guardan outputs de tiempo y altura:
        out = odeint(model,hinicial,[i,i+dt],args=(qin,C,a,b1,g,Lt,b,mu,rho,D,hi))
        vector_tiempo.append(i+dt)
        hinicial = out[:,0][-1]

        #Calculamos caudal, velocidad de salida y perdidas:
        Q = C*a*b1*math.sqrt(g*hi)
        v_2 = 4*Q/(math.pi*D**2)
        hf = 64*(mu/rho)*(Lt/D**2)*(v_2/(2*g))

        #Se calcula la potencia de la iteración:
        P = rho*g*Q*(hi+hinicial-(v_2**2/(2*g))-hf)
        P = efic*P
        Pinicial = P

        #Se guardan los vectores:
        vector_potencias.append(Pinicial) 
        vector_h.append(hinicial)
        vector_qin.append(qin)
        vector_b1.append(b1)
        vector_error.append(error)
        k=k+1

    #Gráficos:
    fig, (ax1,ax2) = plt.subplots(1,2,figsize=(12, 6))
    fig.suptitle('Central hidraulica')
    ax1.plot(vector_tiempo,np.dot(vector_potencias,1/1000000),'b-',linewidth=2,color=color)
    ax1.set_title("Potencia generada [MW]")
    ax2.plot(vector_tiempo,vector_b1,'b-',linewidth=2,color=color)
    ax2.set_title("Apertura de la compuerta [m]")
    
    fig2, (ax3,ax4) = plt.subplots(1,2,figsize=(12, 6))
    fig2.suptitle('Central hidraulica')
    ax3.plot(vector_tiempo,vector_h,'b-',linewidth=2)
    ax3.set_title("Altura del embalse [m]")
    ax4.plot(vector_tiempo,vector_qin,'b-',linewidth=2)
    ax4.set_title("Caudal de entrada [m^3/s]")
   
   

interactive(children=(FloatSlider(value=10.0, description='Pref1', max=80.0), FloatSlider(value=10.0, descript…

Es fácil ver que el controlador logra llegar a todas las potencias de referencia definidas en cada intervalo de tiempo. Luego, vemos que la forma de onda de la apertura de la compuerta es similar a la de la potencia, esto es dado a la ecuación de potencia que llegamos en nuestro modelo. Para esta misma simulación además es útil graficar la altura del embalse. También se establece como caudal de entrada una onda sinusoidal con valor medio $100[m^3/s]$ y amplitud $30[m^3/s]$. Este ejercicio nos sirve para entender el modelo completo de la planta hidrauica y la dinamica que tiene este modelo en cuanto a los cambios de apertura de la compuerta.